In [1]:
import numpy as np
import cv2 
import os
from glob import glob
import matplotlib.pyplot as plt
from keras.applications import ResNet50
from keras.models import Model
import copy
import tensorflow as tf
from tensorflow import keras
import sys
import pandas
from keras_preprocessing.sequence import pad_sequences
import time
from keras.models import Model, Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
# from keras.layers import add
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Flatten,Input, Convolution2D, Dropout, LSTM, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector,Concatenate
from keras.models import Sequential, Model
from keras.layers import concatenate
from keras.layers import Reshape
from keras.layers import (LSTM, BatchNormalization, Dense, Dropout, Embedding,Input, Lambda, TimeDistributed)
from keras import backend as K
from keras import regularizers
from npy_append_array import NpyAppendArray
from tensorflow.keras.utils import to_categorical
import itertools

In [2]:
images_path = 'Flickr_Data/Images/'
images = glob(images_path+'*.jpg')
captions_path = "Flickr_Data/Flickr_TextData/FLickr8k.token.txt"
# captions_path = "results.csv"
max_len = 40

In [3]:
# get sets train, test and dev

train = set()
f = open("Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt", "r")
for line in f:
    train.add(line.strip())
f.close()

test = set()
f = open("Flickr_Data/Flickr_TextData/Flickr_8k.testImages.txt", "r")
for line in f:
    test.add(line.strip())
f.close()

dev = set()
f = open("Flickr_Data/Flickr_TextData/Flickr_8k.devImages.txt", "r")
for line in f:
    dev.add(line.strip())

In [ ]:
# preprocess the images
# generate a dictionary of image filename -> feature vector
# start_index is inclusive

def generate_feature_vectors(start_index, num_of_images, images, model):
    img_feature_vectors = {}

    count = 0
    for item in images[start_index:]:
        img = cv2.imread(item)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        img = img.reshape(1, 224, 224, 3)

        feature_vector = model.predict(img, verbose=0).reshape(2048,)

        img_filename = item.split('\\')[-1]
        img_feature_vectors[img_filename] = feature_vector

        count += 1

        if (count%50==0):
            print(count)

        if (count==num_of_images):
            break

    return img_feature_vectors

In [ ]:
# makes a string lowercase, prepends it with the string 'sos' and appends with 'eos'
def process_string(s):
    s = s.lower()
    s = 'sos ' + s + ' eos'
#     s = s.replace(" a ", " ")
    return s

In [ ]:
# preprocess the captions data

# generate a dictionary of image filename -> list of captions

def generate_captions_dict(captions_path, img_feature_vectors):
    captions_dict = {}

    f = open(captions_path, 'r').read().split('\n')

    # generate a dictionary of filenames to a list of captions
    for line in f:
        try:
            filename_caption = line.split('\t') 
            filename = filename_caption[0][:-2]
            caption = process_string(filename_caption[1])

            if filename in img_feature_vectors:
                if filename not in captions_dict:
                    captions_dict[filename] = [caption]
                else:
                    captions_dict[filename].append(caption)
        except:
            pass
        
    return captions_dict

In [ ]:
# for flickr30k

def generate_captions_dict(captions_path, img_feature_vectors):
    captions_dict = {}
    
    f = open(captions_path, 'r', encoding='utf8')
    for line in f:
        try:
            filename_caption = line.split("|")
            filename = filename_caption[0]
            caption = process_string(filename_caption[2].strip())
            
            if filename in img_feature_vectors:
                if filename not in captions_dict:
                    captions_dict[filename] = [caption]
                else:
                    captions_dict[filename].append(caption)
        except:
            pass
        
    f.close()
    return captions_dict
        

In [ ]:
# generates a list of all captions in a text file 

# def get_list_of_captions(captions_path):
#     f = open(captions_path, 'r').read().split('\n')
#     all_captions = []

#     for line in f:
#         try:
#             filename_caption = line.split('\t') 
#             caption = process_string(filename_caption[1])
#             all_captions.append(caption)
#         except:
#             pass
        
#     return all_captions

In [ ]:
# generate a dictionary called vocab which will contain all words in the list of captions mapped to a unique integer
# does not include words that appear less than n times
# only keeps common words

# def create_common_vocab(captions_path, n):
#     vocab_freq = {}
#     vocab_dict = {}
#     all_captions = get_list_of_captions(captions_path)
#     count = 1
#     for caption in all_captions:
#         caption_as_list = caption.split()
#         for word in caption_as_list:
#             if word not in vocab_freq:
#                 vocab_freq[word] = 1
#             else:
#                 vocab_freq[word] = vocab_freq[word]+1
#     vocab_list = [w for w in vocab_freq if vocab_freq[w] >= n]
#     count = 1
#     for word in vocab_list:
#         if word not in vocab_dict:
#             vocab_dict[word] = count
#             count+=1
#     return vocab_dict

In [ ]:
def create_common_vocab(captions_dict, n):
    vocab_freq = {}
    vocab_dict = {}
    for key, value in captions_dict.items():
        for caption in value:
            caption_as_list = caption.split()
            for word in caption_as_list:
                if word not in vocab_freq:
                    vocab_freq[word] = 1
                else:
                    vocab_freq[word] = vocab_freq[word]+1
    vocab_list = [w for w in vocab_freq if vocab_freq[w] >= n]
    count = 1
    for word in vocab_list:
        if word not in vocab_dict:
            vocab_dict[word] = count
            count+=1
    return vocab_dict

In [ ]:
# takes a string and returns a list of integers where each integer corresponds to a particular word
def encode_string(s, vocab):
    s_list = s.split()
    encoded_string = []
    for word in s_list:
            if word in vocab:
                encoded_string.append(vocab[word])
    return encoded_string

In [ ]:
# generates a copy of captions_dict where each caption is replaced with a list of integers where each integer corresponds to a word in the caption

def encode_captions_dict(captions_dict, vocab):
    captions_dict_encoded = copy.deepcopy(captions_dict)

    for filename, captions in captions_dict_encoded.items():
        for i, caption in enumerate(captions):
            captions[i] = encode_string(caption, vocab)
    return captions_dict_encoded

In [ ]:
# get length of the longest caption in the data set
def get_max_length(captions_dict):
    max_len = 0
    for key, value in captions_dict.items():
        for caption in value:
            if len(caption.split())>max_len:
                max_len = len(caption.split())
    return max_len

# max_len = 40

In [20]:
# batch size -> number of images will data be generated for at once

def generate_training_data(image_feature_vectors, captions_dict_encoded, max_len, vocab_len, batch_size, num_of_images):
    X_file = 'X_dev.npy'
    y_in_file = 'y_in_dev.npy'
    y_out_file = 'y_out_dev.npy'
    filenames_file = 'filenames_dev.npy'
    
    X = []
    y_in = []
    y_out = []
    filenames = set()
    
    n = 0
    for filename, captions in captions_dict_encoded.items():
        n+=1
#         if n <= 6473:
#             continue
        for caption in captions:
            i = 0
            for word in caption:
                if i==0:
                    i+=1
                    continue
                y_in_item = [caption[:i]]
                y_in_item = pad_sequences(y_in_item, maxlen=max_len, truncating='post')[0]
                y_in.append(y_in_item)

                y_out_item = to_categorical([word], num_classes=vocab_len+1)[0]
                y_out.append(y_out_item)

                X_item = image_feature_vectors[filename]
                X.append(X_item)
                
                filenames.add(filename)
                i+=1
        if n%batch_size==0 or n==num_of_images:
            print(n)
            # shuffle rows before saving
            with NpyAppendArray(X_file) as npaa:
                npaa.append(np.array(X))
            with NpyAppendArray(y_in_file) as npaa:
                npaa.append(np.array(y_in))
            with NpyAppendArray(y_out_file) as npaa:
                npaa.append(np.array(y_out))
            X = []
            y_in = []
            y_out = []
            if n == num_of_images:
                break
    np.save(filenames_file, filenames)

In [ ]:
resnet_model = ResNet50(include_top=True)
# restructure model
resnet_model = Model(inputs=resnet_model.input, outputs=resnet_model.layers[-2].output)

img_feature_vectors = generate_feature_vectors(0, 8091, images, resnet_model)

In [ ]:
captions_dict = generate_captions_dict(captions_path, img_feature_vectors)
vocab = create_common_vocab(captions_dict, 10)
captions_dict_encoded = encode_captions_dict(captions_dict, vocab)

In [ ]:
# np.save('img_feature_vectors.npy', img_feature_vectors)
# np.save('captions_dict.npy', captions_dict)
# np.save('captions_dict_encoded.npy', captions_dict_encoded)
# np.save('vocab.npy', vocab)

In [4]:
# img_feature_vectors = np.load('img_feature_vectors.npy', allow_pickle='TRUE').item()
# captions_dict_encoded = np.load('captions_dict_encoded.npy', allow_pickle='TRUE').item()
vocab = np.load("vocab.npy", allow_pickle='TRUE').item()
# embedding_matrix = np.load('embedding_matrix.npy')

In [3]:
captions_dict_encoded_dev = {key: captions_dict_encoded[key] for key in dev}

NameError: name 'dev' is not defined

In [24]:
start_time = time.time()

vocab_len = len(vocab)

generate_training_data(img_feature_vectors, captions_dict_encoded_dev, 40, vocab_len, 1000, 1000)

print(time.time()-start_time)

1000
5.8042988777160645


Word Embeddings and other stuff

In [6]:
word_embeddings = {}
f = open("glove/glove.6B.200d.txt", encoding="utf-8")
for line in f:
    word_vector = line.split()
    word = word_vector[0]
    vector = np.asarray(word_vector[1:], dtype='float32')
    word_embeddings[word] = vector
f.close()

In [8]:
vocab = np.load("vocab.npy", allow_pickle=True).item()

In [50]:
word_embeddings.get("t-shirt")

array([-0.013748 , -0.28352  , -0.33465  ,  0.29837  ,  0.61233  ,
        0.60097  ,  0.4735   ,  0.42694  ,  0.32204  , -0.22364  ,
       -0.27225  ,  0.085531 , -0.1714   ,  0.48952  , -0.062202 ,
       -0.66706  , -0.54124  ,  0.37751  ,  0.042753 , -0.0039544,
        0.483    , -0.45625  ,  0.35832  , -0.1135   , -0.013117 ,
       -1.1562   ,  0.092714 ,  0.1407   , -0.22857  ,  0.087142 ,
        0.31291  ,  0.11429  ,  0.1032   ,  0.18801  ,  0.21747  ,
       -0.010851 ,  0.095892 , -0.40989  , -0.17735  , -0.33253  ,
        0.08932  , -0.019247 , -0.039571 ,  0.12407  ,  0.69643  ,
       -0.44594  ,  0.42551  , -0.019997 , -0.058083 ,  0.14584  ,
        0.12179  , -0.18677  ,  0.44433  ,  0.21295  ,  0.001713 ,
        0.15448  , -0.82974  , -0.29175  , -0.038278 , -0.68382  ,
       -0.62029  , -0.30044  , -0.11898  ,  0.15675  , -0.49229  ,
        0.37115  , -0.28078  , -0.37748  ,  0.34853  ,  0.087878 ,
        0.37517  , -0.61631  ,  0.35614  , -0.53103  , -0.0430

In [7]:
embedding_size = 200
vocab_size = 1965+1
# word to index

embedding_matrix = np.zeros((vocab_size, embedding_size))

for word, i in vocab.items():
    #if i < max_words:
    embedding_vector = word_embeddings.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [8]:
np.save('embedding_matrix.npy', embedding_matrix)

In [ ]:
#version 1

embedding_size = 128
vocab_len = 3988+1
max_len = 40

#image
x = keras.Sequential([
    keras.layers.Input(shape=(2048,)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.RepeatVector(max_len)
])

#caption
y = keras.Sequential([
    keras.layers.Embedding(vocab_len, embedding_size, input_length=max_len),
    keras.layers.LSTM(256, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(embedding_size))
])

#concatinate outputs of y and x
z = keras.layers.Concatenate()([x.output, y.output])
z = keras.layers.LSTM(128, return_sequences=True)(z)
z = keras.layers.LSTM(512, return_sequences=False)(z)
z = keras.layers.Dense(vocab_len, activation='softmax')(z)

model = Model(inputs=[x.input, y.input], outputs=z)

model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model.summary()

In [ ]:
#version 2

vocab_size = 1965+1
max_len = 40
embedding_size = 200

# image feature extractor model
image_input = Input(shape=(2048,))
ii1 = Dropout(0.5)(image_input)
ii2 = Dense(256, activation='relu')(ii1)

# partial caption sequence model
caption_input = Input(shape=(max_len,))
ci1 = Embedding(vocab_size, embedding_size, mask_zero=True)(caption_input)
ci2 = Dropout(0.5)(ci1)
ci3 = LSTM(256)(ci2)

# decoder (feed forward) model
output = add([ii2, ci3])
output = Dense(256, activation='relu')(output)
output = Dense(vocab_size, activation='softmax')(output)

# merge the two input models
model = Model(inputs=[image_input, caption_input ], outputs=output)